# Preprocessing

In het vorige vak heb je reeds gezien hoe je met behulp van python en pandas de dataset kan transformeren naar een bruikbaarder formaat.
Daarbij hadden we ook gezien dat dit kon toegevoegd worden in een pipeline zodat alles binnen het SkLearn platform gehouden kon worden.
Het voordeel van het opstellen van een pijplijn is dat alle nodige preprocessing stappen mee opgeslagen en gedeployed worden.
Dus moet de nodige code voor het voorbereiden van de data niet overgezet worden van de trainingscode naar de productiecode.
Dit vermijdt bugs en zorgt voor een kleiner verschil tussen de performantie van het model in het labo/development en productie.

Het is dus duidelijk dat het gebruik van een pijplijn voor je data om te zetten naar een voorspelling een belangrijk concept is binnen Machine Learning/Data Science.
Hierdoor is het dus evident dat dit concept ook binnen tensorflow mogelijk is.
De volgende opties worden hiervoor aangeboden:
* Door de preprocessing stappen/lagen onderdeel te maken van het model
    * Via het sequential model 
    * Via de function API
* Door gebruik te maken van het Dataset object concept binnen tensorflow om de preprocessing stappen eerst te doen en de kant en klare batches aan het model te presenteren.

Er zijn twee manieren om de preprocessing stappen deel te laten uitmaken van het model.
Ten eerste is het nodig om je te herinneren dat een model in tensorflow bestaat uit een aantal lagen die achter elkaar uitgevoerd worden.
Deze laggen kunnen voorsteld worden als een graaf. 

Er zijn twee manieren om om te gaan met deze lagen.
Ten eerste kan je gebruik maken van een sequentie model.
Hierbij moeten al je inputs op dezelfde manier behandeld worden.
Dit kan bijvoorbeeld gebeuren als je data reeds gepreprocessed is of als het enkel numerieke waarden bevat.
Ook als het volledige uit tekst bestaat (zoals een tweet) kan je dit model gebruiken.
Indien je echter complexere zaken wilt doen zoals een set features normalizeren en een andere set features encoderen met one-hot encoding, dan moet je gebruik maken van de functional API om een acyclische graaf op te stellen van de uit te voeren stappen.

## With Sequential model

Laten we in eerste instantie eerst kijken naar hoe je de verschillende preprocessing stappen kan gebruiken in het sequentieel model.

**Numerieke data**

Om te werken met numerieke data zijn er de volgende mogelijkheden:
* Normalizatie: zet het om naar een standaardverdeling met gemiddelde 0 en standaardafwijking 1
* Discretizatie: zet continue waarden om naar discrete/gehele getallen/categorieën

Hieronder staat een voorbeeld hoe deze lagen kunnen toegepast worden op een dataset met numeriek gegevens (pixelwaarden) van 50.000 figuren van 32 pixels op 32. Let op dat voor je dit door een normalizatie laag kunt sturen, dat je ervoor moet zorgen dat de tensor vlak is.
Dit wil zeggen dat de tensor slechts uit 1 dimensie bestaat.

Deze aangemaakte lagen kunnen dat toegevoegd worden aan een sequentieel model als volgt

**Categorieke data**

De preprocessing stappen die vaak uitgevoerd worden op categorieke data hebben te maken met hun encoding.
Zo is het bijvoorbeeld nodig om:
* Een numerieke encoding om te zetten naar one-hot, multi-hot of andere voorstellingen. Dit kan met de CategoryEncoding laag.
* Hashing uit te voeren op een categorieke feature om een vector te bekomen
* Een tekstuele categorieke waarde te encoderen zodat het door een neuraal netwerk gelezen kan worden
*  Een integer waarde om te zetten naar een representatie dat gelezen kan worden door een neuraal netwerk

**Welke kolom wordt gebruikt voor de onbekenden?**

**Tekstuele data**

Hoe we tot nu toe gewerkt hebben met tekstuele data (zoals een tweet) was om er een bag of words van te maken.
Op basis van deze bag of words kon dan ook een tf-idf waarde berekend worden om rekening te houden met heel vaak voorkomende woorden.
Bij tensorflow kan je iets gelijkaardigs doen door gebruik te maken van de **TextVectorization** laag.
Deze laag krijgt een volledige string mee en kan deze standaardizeren of splitsen op basis van een karakter.
Standaardizeren van een string houdt in dat extra spaties en leestekens verwijderd worden of dat alle hoofdletters omgezet worden naar kleine letters. Als je een andere zaken wil bereiken kan je ook een functie meegeven die opgeroepen wordt voor elke string.

Verder kan de TextVectorization laag ook een bag-of-words berekenen of zelfs een tf-idf waarde berekenen.

**Preprocessing van figuren**

Aangezien computervisie een heel belangrijke toepassing is, zijn er een heel breed spectrum van lagen voorzien om te werken met beelden.
Bij Data Augmentation worden er ook nog lagen aangehaald die in principe onder preprocessing lagen vallen maar nu nog minder relevant zijn.
Een aantal andere belangrijke lagen zijn:
* Resizing()
* Rescaling()
* Flatten()
* ...

Een voorbeeld voor te werken met dit soort data wordt getoond onder het Computer Visie gedeelte .

## With Functional API

Het probleem om te werken met het sequentiele model is dat de inputs allemaal van hetzelfde type moeten zijn. 
Dit houdt in dat ze allemaal numeriek of categoriek moeten zijn.
In veel gevallen is dit geen probleem, bijvoorbeeld bij het verwerken van tweets of figuren. 
Indien je werkt met gestructureerde data is dit echter vaak te beperkend.
Om deze reden kan je ook gebruik maken van de functional API.
Zet hierbij het voorbeeld van de numerieke data om zodat het werkt met de functional API ipv het sequentiele model.
Bekijk hiervoor ook de code op het einde van de notebook van vorige les.

# Computer visie

Het tweede deel van deze notebook staat in het teken van computer visie.
Dit deel van Machine Learning is tegenwoordig heel populair en behaalt door middel van Deep Learning neurale netwerken heel goede resultaten bij het interpreteren van beelden en video.
De toepassingen van Computer visie zijn omvangrijk, bijvoorbeeld:
* Medische wereld: interpreteren scans, robots om te helpen bij kinesie therapie
* Mobiliteit: Zelfrijdende auto's
* Productie: Magazijnen waar robots zelf items halen of kijken hoe ze iets in elkaar moeten steken
* Gaming: Geavanceerde bots
* Media: Maken en detecteren van deep fakes/misinformatie

In deze domeinen wordt computervisie gebruikt zowel als regressie en classificatie techniek.
Daarnaast zijn ook transformers veel voorkomend. Transformers zijn neurale netwerken die een beeld omzetten in een ander beeld. 
Dit wordt bijvoorbeeld gebruikt voor image segmentation waarbij je kan kijken waar in de figuur welke objecten aanwezig zijn om daarop analyses te doen.

Hier gaan we werken met een standaard classificatieprobleem binnen het domein van computervisie, namelijk de CIFAR10-dataset.
Deze dataset bestaat uit 60000 32x32 kleurbeelden (50000 trainingsdata, 10000 testdata). 
Er zijn 10 mogelijke klassen in deze dataset met 6000 beelden per klasse.
De mogelijke klassen zijn:
* airplane 
* automobile 
* bird 
* cat 
* deer 
* dog 
* frog 
* horse 
* ship 
* truck

Deze oefening gaan we uitwerken in drie stappen:
* Data genereren
* Data augmentation
* Opstellen neuraal netwerk

## Data generation

Laten we beginnen met deze dataset te downloaden van kaggle en te unzippen. Let er hierbij op dat de trainings- en testdata in verschillende mappen geplaatst worden.

In theorie kan je nu alle beelden inladen en gebruiken als data.
Echter is dit snel een grote belasting voor het geheugen van je computer.
Hierdoor kan het beter zijn om de data slechts in stukjes in te lezen.
Dit gebeurd in de code hieronder.

In [2]:
# download
import opendatasets as od
od.download("https://www.kaggle.com/competitions/cifar-10/overview")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jensbaetensodisee
Your Kaggle Key: ········


100%|███████████████████████████████████████████████████████████████████████████████| 715M/715M [00:34<00:00, 21.7MB/s]



Extracting archive .\cifar-10/cifar-10.zip to .\cifar-10


In [18]:
# install for handling 7z
!pip install pyunpack
!pip install patool
!pip install scikit-image


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip
     --------------------------------------- 12.0/12.0 MB 27.3 MB/s eta 0:00:00
     ---------------------------------------- 3.4/3.4 MB 35.7 MB/s eta 0:00:00
     ---------------------------------------- 4.2/4.2 MB 33.3 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 32.5 MB/s eta 0:00:00
     ------------------------------------- 208.5/208.5 kB 13.2 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Data augmentation

Het verzamelen van voldoende data voor het trainen van Deep Learning computervisie toepassingen kan heel kostelijk worden omdat er vaak heel veel figuren (tienduizenden tot miljoenen) moeten verzameld en gelabeld worden.
Het is dus duidelijk dat dit niet ideaal is.

Een mogelijkheid om de benodigde hoeveelheid voorbeelden/data/figuren te beperken is door kleine wijzigingen aan te brengen aan je bestaande data.
Hierdoor zorg je voor meer variatie wat er voor zorgt dat je model beter kan generaliseren en de klans op overfitting verkleint.
Hierdoor krijg je typisch betere resultaten.
Een voorbeeld hoe je dit kan doen, zie je in onderstaande code.
Let op dat het de bedoeling is dat deze code uiteindelijk deel uitmaakt van de totale pipeline maar hier houden we het nog apart om het resultaat van de augmentatie stappen te kunnen bepalen.

## Convolutionele neurale netwerken

### Convolutionele lagen

### Pooling lagen

### Full example